In [1]:
# read data

from mnist import MNIST
import numpy as np
mndata = MNIST("/Users/jigyayadav/Desktop/Codes/neuralnets253/HW1")
mndata.gz = True
images, labels = mndata.load_training() #Images is a list of 60000 images of 784 dimensions, Labels is a list of 60000 ints
imagesTest, labelsTest = mndata.load_testing()

images = np.array(images)
labels = np.array(labels)
imagesTest = np.array(imagesTest)
labelsTest = np.array(labelsTest)

images_train = images[:200]
images_test = imagesTest[-50:]
labels_train = labels[:200]
labels_test = labelsTest[-50:]
images_train = np.array(images_train)
labels_train = np.array(labels_train)
images_test = np.array(images_test)
labels_test = np.array(labels_test)

# Normalization
images_train = images_train*(1/127.5)
images_test = images_test*(1/127.5)
arrOnes = np.ones_like(images_train)
images_train = images_train-arrOnes
arrOnes = np.ones_like(images_test)
images_test = images_test-arrOnes

# Should this be appended after or before normalization
images_train = np.insert(images_train, 0, 1, axis=1)
images_test = np.insert(images_test, 0, 1, axis=1)

numFeatures = len(images[0, :])

# Divide between validation and training
from sklearn.model_selection import train_test_split
images_train, images_validation, labels_train, labels_validation = train_test_split(images_train, labels_train, test_size=0.10)

In [6]:
labelNums = 10

def labelsToVectForm(labels):
    vectForm = np.zeros((len(labels), labelNums))
    for i in range(len(labels)):
        vectForm[i][labels[i]] = 1
    return vectForm

t_train = labelsToVectForm(labels_train)
t_test = labelsToVectForm(labels_test)
t_validation = labelsToVectForm(labels_validation)

x = images_train
t = t_train

In [17]:
t.shape

(180, 10)

In [31]:
from sklearn.metrics import accuracy_score
import math

# Forward propagation

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmaxFunc(prod):
    prod = np.exp(prod)
    for i in range(len(prod)):
        rowSum = sum(prod[i, :])
        for j in range(len(prod[i, :])):
            prod[i][j] = prod[i][j]/rowSum    
    return prod

def getPredClass(y):
    predClass = np.argmax(y, axis=1)
    return predClass

def calculateOutput(x, W1, W2):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)
    X2 = np.insert(Z1, 0, 1, axis=1)
    A2 = X2.dot(W2)
    Y = softmaxFunc(A2)
    return Y

def calculateCost(x, t, W1, W2):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -1*(np.sum(np.multiply(t, np.log(Y))))/sampLen
    return cost

def calculateCostOneInstance(x, t, W1, W2, n):
    sampLen = len(x)
    Y = calculateOutput(x, W1, W2)
    cost = -1*(np.multiply(t, np.log(Y)))/sampLen
    return np.sum(cost[n, :])

def calculateAccuracy(x, labels, W1, W2):
    Y = calculateOutput(x, W1, W2)
    predClass = getPredClass(Y)
    return accuracy_score(labels, predClass)


W1 = np.random.rand(785, 64)
W2 = np.random.rand(65, 10)
epsilon = 0.001
learningRate = 0.1

for itr in range(100):
    A1 = x.dot(W1)
    Z1 = sigmoid(A1)

    X2 = np.insert(Z1, 0, 1, axis=1)
    A2 = X2.dot(W2)
    Y = softmaxFunc(A2)
    print(Y)

    # Backward propagation
    W2_nobias = np.delete(W2, (0), axis=0)
    summation_k = (Y-t).dot(W2_nobias.T)
    g_derivative = Z1*(1-Z1)
    delE_Wij = x.T.dot(np.multiply(summation_k, g_derivative))
    delE_Wij = delE_Wij*(1/len(x))
    
    # Check approximate derivative
    for n in range(0, 150, 30):
        print("For input to hidden layers")
        for i in range(0, 785, 100):
            for j in range(0, 64, 20):
                del_En_Wij = (g_derivative[n][j]*summation_k[n][j]*x[n][i])/len(x)
                modifiedW1 = W1
                modifiedW1[i][j] += epsilon
                costFunction1 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
                modifiedW1[i][j] -= 2*epsilon
                costFunction2 = calculateCostOneInstance(x, t, modifiedW1, W2, n)
                approx_delEn = (costFunction1-costFunction2)/(2*epsilon)
                diff = abs(del_En_Wij-approx_delEn)
                print(del_En_Wij, " ", approx_delEn)
                print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))
        
        print("For hidden to output layers")
        for j in range(0, 65, 20):
            for k in range(0, 10, 3):
                actualDerivative = X2[n][j]*(Y[n][k]-t[n][k])
                modifiedW2 = W2
                modifiedW2[j][k] += epsilon
                costFunction1 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
                modifiedW2[j][k] -= 2*epsilon
                costFunction2 = calculateCostOneInstance(x, t, W1, modifiedW2, n)
                approximateDerivative = (costFunction1-costFunction2)/(2*epsilon)
                diff = abs(actualDerivative-approximateDerivative)
                print(actualDerivative, " ", approximateDerivative)
                print("Difference between approximate gradient = ", diff, " ", abs(epsilon*epsilon-diff))
                
    W2 = W2-learningRate*((X2.T.dot(Y-t))/len(x))
    W1 = W1-learningRate*(delE_Wij)
    
    # Calculate validation error
    currTrainingError = calculateCost(x, t, W1, W2)
    currvalidationError = calculateCost(images_validation, t_validation, W1, W2)
    currTrainingAccuracy = calculateAccuracy(x, labels_train, W1, W2)
    currValidationAccuracy = calculateAccuracy(images_validation, labels_validation, W1, W2)
    print("Current training error = ", currTrainingError)
    print("Current validation error = ", currvalidationError)
    print("Current training accuracy = ", currTrainingAccuracy)
    print("Current validation accuracy = ", currValidationAccuracy)
    

[[0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]
 [0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]
 [0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]
 ...
 [0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]
 [0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]
 [0.07558652 0.07052174 0.12254478 ... 0.14992883 0.12692533 0.064885  ]]
For input to hidden layers
5.705113406408025e-136   0.0
Difference between approximate gradient =  5.705113406408025e-136   1e-06
-2.000895450815477e-139   0.0
Difference between approximate gradient =  2.000895450815477e-139   1e-06
1.6041571113226919e-131   0.0
Difference between approximate gradient =  1.6041571113226919e-131   1e-06
-2.6895478576400388e-136   0.0
Difference between approximate gradient =  2.6895478576400388e-136   1e-06
-5.705113406408025e-136   0.0
Difference between approximate gradient =  5.705113406408025e-136   1e-06
2.00089

2.2364383951829374e-129   0.0
Difference between approximate gradient =  2.2364383951829374e-129   1e-06
3.8133496988901887e-129   0.0
Difference between approximate gradient =  3.8133496988901887e-129   1e-06
1.964830331523124e-129   0.0
Difference between approximate gradient =  1.964830331523124e-129   1e-06
-2.7667988948589975e-128   0.0
Difference between approximate gradient =  2.7667988948589975e-128   1e-06
For input to hidden layers
9.056200006814772e-103   0.0
Difference between approximate gradient =  9.056200006814772e-103   1e-06
5.173793729400174e-99   0.0
Difference between approximate gradient =  5.173793729400174e-99   1e-06
-3.3124337482562857e-102   0.0
Difference between approximate gradient =  3.3124337482562857e-102   1e-06
-4.667882187456373e-103   0.0
Difference between approximate gradient =  4.667882187456373e-103   1e-06
-9.056200006814772e-103   0.0
Difference between approximate gradient =  9.056200006814772e-103   1e-06
-5.173793729400174e-99   0.0
Differe

-1.1741932240184917e-139   0.0
Difference between approximate gradient =  1.1741932240184917e-139   1e-06
For hidden to output layers
0.0755865235700809   0.0004190888063622042
Difference between approximate gradient =  0.0751674347637187   0.0751664347637187
0.12888253372726893   0.0007146419260858211
Difference between approximate gradient =  0.1281678918011831   0.1281668918011831
0.06640678969059383   0.00036826696685806487
Difference between approximate gradient =  0.06603852272373575   0.06603752272373575
0.06488499602589476   0.00035985151631157813
Difference between approximate gradient =  0.06452514450958319   0.06452414450958319
4.4144756673917976e-133   0.0
Difference between approximate gradient =  4.4144756673917976e-133   1e-06
7.527119679784241e-133   0.0
Difference between approximate gradient =  7.527119679784241e-133   1e-06
3.878352163754859e-133   0.0
Difference between approximate gradient =  3.878352163754859e-133   1e-06
3.789474930270554e-133   0.0
Difference be

2.0421068479341718e-139   0.0
Difference between approximate gradient =  2.0421068479341718e-139   1e-06
-1.6366926334197284e-131   0.0
Difference between approximate gradient =  1.6366926334197284e-131   1e-06
2.744357542176446e-136   0.0
Difference between approximate gradient =  2.744357542176446e-136   1e-06
5.8387026954591885e-136   0.0
Difference between approximate gradient =  5.8387026954591885e-136   1e-06
-2.0421068479341718e-139   0.0
Difference between approximate gradient =  2.0421068479341718e-139   1e-06
1.6366926334197284e-131   0.0
Difference between approximate gradient =  1.6366926334197284e-131   1e-06
-2.744357542176446e-136   0.0
Difference between approximate gradient =  2.744357542176446e-136   1e-06
-5.8387026954591885e-136   0.0
Difference between approximate gradient =  5.8387026954591885e-136   1e-06
2.0421068479341718e-139   0.0
Difference between approximate gradient =  2.0421068479341718e-139   1e-06
-1.6366926334197284e-131   0.0
Difference between appro

8.8090207610237555e-103   0.0
Difference between approximate gradient =  8.8090207610237555e-103   1e-06
5.020037867557879e-99   0.0
Difference between approximate gradient =  5.020037867557879e-99   1e-06
-3.214441020097783e-102   0.0
Difference between approximate gradient =  3.214441020097783e-102   1e-06
-4.532281150488955e-103   0.0
Difference between approximate gradient =  4.532281150488955e-103   1e-06
-9.168572628820644e-103   0.0
Difference between approximate gradient =  9.168572628820644e-103   1e-06
-5.22493737235616e-99   0.0
Difference between approximate gradient =  5.22493737235616e-99   1e-06
3.3456426943874887e-102   0.0
Difference between approximate gradient =  3.3456426943874887e-102   1e-06
4.717272217855851e-103   0.0
Difference between approximate gradient =  4.717272217855851e-103   1e-06
-9.168572628820644e-103   0.0
Difference between approximate gradient =  9.168572628820644e-103   1e-06
-5.22493737235616e-99   0.0
Difference between approximate gradient = 

...

-0.9343997677417721   -0.005191254940322201
Difference between approximate gradient =  0.9292085128014499   0.9292075128014499
1.1480772786096586e-127   0.0
Difference between approximate gradient =  1.1480772786096586e-127   1e-06
1.9170706814256736e-127   0.0
Difference between approximate gradient =  1.9170706814256736e-127   1e-06
1.0078761201369308e-127   0.0
Difference between approximate gradient =  1.0078761201369308e-127   1e-06
-1.419079156072925e-126   0.0
Difference between approximate gradient =  1.419079156072925e-126   1e-06
1.3932824597370842e-128   0.0
Difference between approximate gradient =  1.3932824597370842e-128   1e-06
2.3265166938424763e-128   0.0
Difference between approximate gradient =  2.3265166938424763e-128   1e-06
1.223137279988008e-128   0.0
Difference between approximate gradient =  1.223137279988008e-128   1e-06
-1.722164643419568e-127   0.0
Difference between approximate gradient =  1.722164643419568e-127   1e-06
2.2367105745557094e-129   0.0
Differe

7.408666438471531e-136   0.0
Difference between approximate gradient =  7.408666438471531e-136   1e-06
-1.2138960154528192e-139   0.0
Difference between approximate gradient =  1.2138960154528192e-139   1e-06
8.034921363738037e-139   0.0
Difference between approximate gradient =  8.034921363738037e-139   1e-06
2.6527470481679667e-139   0.0
Difference between approximate gradient =  2.6527470481679667e-139   1e-06
7.408666438471531e-136   0.0
Difference between approximate gradient =  7.408666438471531e-136   1e-06
-1.2138960154528192e-139   0.0
Difference between approximate gradient =  1.2138960154528192e-139   1e-06
For hidden to output layers
0.07559572261290705   0.00041913733411086396
Difference between approximate gradient =  0.07517658527879618   0.07517558527879618
0.12623047782802163   0.000699932389046018
Difference between approximate gradient =  0.12553054543897563   0.12552954543897563
0.06636410721263744   0.00036802712080185457
Difference between approximate gradient =  

2.2157801511092483e-139   0.0
Difference between approximate gradient =  2.2157801511092483e-139   1e-06
-1.7735908951205328e-131   0.0
Difference between approximate gradient =  1.7735908951205328e-131   1e-06
2.9752332668440465e-136   0.0
Difference between approximate gradient =  2.9752332668440465e-136   1e-06
-6.402676693693932e-136   0.0
Difference between approximate gradient =  6.402676693693932e-136   1e-06
2.2157801511092483e-139   0.0
Difference between approximate gradient =  2.2157801511092483e-139   1e-06
-1.7735908951205328e-131   0.0
Difference between approximate gradient =  1.7735908951205328e-131   1e-06
2.9752332668440465e-136   0.0
Difference between approximate gradient =  2.9752332668440465e-136   1e-06
-6.402676693693932e-136   0.0
Difference between approximate gradient =  6.402676693693932e-136   1e-06
2.2157801511092483e-139   0.0
Difference between approximate gradient =  2.2157801511092483e-139   1e-06
-1.7735908951205328e-131   0.0
Difference between appro

3.715972937394791e-129   0.0
Difference between approximate gradient =  3.715972937394791e-129   1e-06
1.9631748718808618e-129   0.0
Difference between approximate gradient =  1.9631748718808618e-129   1e-06
-2.764166810563256e-128   0.0
Difference between approximate gradient =  2.764166810563256e-128   1e-06
For input to hidden layers
9.630067349374677e-103   0.0
Difference between approximate gradient =  9.630067349374677e-103   1e-06
5.434100818379653e-99   0.0
Difference between approximate gradient =  5.434100818379653e-99   1e-06
-3.4816504648702985e-102   0.0
Difference between approximate gradient =  3.4816504648702985e-102   1e-06
-4.920591038879119e-103   0.0
Difference between approximate gradient =  4.920591038879119e-103   1e-06
-9.630067349374677e-103   0.0
Difference between approximate gradient =  9.630067349374677e-103   1e-06
-5.434100818379653e-99   0.0
Difference between approximate gradient =  5.434100818379653e-99   1e-06
3.4816504648702985e-102   0.0
Difference 

KeyboardInterrupt: 